# How to Display Bounding Boxes on Objects in a Video.

One of the most common requirements of computer vision applications is detecting objects in images and displaying bounding boxes around those objects. In this cookbook we'll walk through the steps on how to utilize the open source Roboflow ecosystem to acomplish this task on a video. Let's dive in! 

## Download A Video

First, let's download a video that we can detect objects in. Let's use `curl` to install a video of people walking that Roboflow hosts. We'll then disply the video with the `Video()` function imported from `IPython.display` to see what we're working with.

In [1]:
!curl -O https://media.roboflow.com/supervision/video-examples/people-walking.mp4

from IPython.display import Video
Video(filename="people-walking.mp4")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7428k  100 7428k    0     0  5429k      0  0:00:01  0:00:01 --:--:-- 5430k


## Detecting Objects

In order to display bounding boxes on objects, we first need a way to detect those objects. We'll be using the open source [Inference](https://github.com/roboflow/inference) package for this task. Inference allows us to quickly use thousands of models, including fine tuned models from [Roboflow Universe](https://universe.roboflow.com/), in a few lines of code.  Along with this, we'll be utilizing some utilities for working with our video data from the [Supervision](https://github.com/roboflow/supervision) package. Let's install both packages in a virtual environment with `pip`.

In [2]:
#!/bin/bash
!python -m venv venv
!source venv/bin/activate
!pip install inference supervision

Now that we've got our packages installed. Let's talk through the code to display bounding boxes on objects. 

In [3]:
import supervision as sv
from inference.models.utils import get_roboflow_model


if __name__ == "__main__":

    model = get_roboflow_model("yolov8n-640")
    video_name = "people-walking.mp4"
    frame_generator = sv.get_video_frames_generator(video_name)
    video_info = sv.VideoInfo.from_video_path(video_name)

    box_annotator = sv.BoundingBoxAnnotator()

    with sv.VideoSink(target_path="output.mp4", video_info=video_info) as sink:
        for frame in frame_generator:
            result = model.infer(frame)[0]
            detections = sv.Detections.from_inference(result)
            annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)
            sink.write_frame(frame=annotated_frame)



/opt/homebrew/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


In [6]:
!open output.mp4